In [11]:
import os
import pandas as pd
from tqdm import tqdm
import pickle


In [12]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r.pickle", "rb") as f:
    nutids_r = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [13]:
import random

step_rates = [1, 2, 3]
pre_pad_result = []
len_of_list = int(len(lines))
for i in tqdm(range(0, len_of_list//3,step_rates[0])):
    pre_pad_result.append((' '.join(lines[i:i+step_rates[0]])).split())
for i in tqdm(range(len_of_list//3, 2*len_of_list//3,step_rates[1])):
    pre_pad_result.append((' '.join(lines[i:i+step_rates[1]])).split())

random.shuffle(pre_pad_result)

100%|██████████| 328134/328134 [00:27<00:00, 11830.82it/s] 


In [14]:
scope = 10
middle = int(scope/2)
padding = int(scope/2)

In [15]:
pre_cleaned_result = [["<PAD>"]*padding + lst + ["<PAD>"]*padding for lst in tqdm(pre_pad_result)]

100%|██████████| 984400/984400 [00:39<00:00, 24998.00it/s] 


In [16]:
char = "*@;:!\"?«».,"
post_pad_result = [[word.translate(str.maketrans('', '', ''.join(char))) for word in pre_cleaned_lst] for pre_cleaned_lst in tqdm(pre_cleaned_result)]

# for i in tqdm(range(len(pre_cleaned_result))):
#     pre_cleaned_lst = pre_cleaned_result[i]
#     lst = [word.translate(str.maketrans('', '', ''.join(char))) for word in pre_cleaned_lst]
#     post_pad_result.append(lst)

100%|██████████| 984400/984400 [01:21<00:00, 12106.52it/s]


In [17]:
saved_post_pad_result = post_pad_result

In [18]:
post_pad_result = saved_post_pad_result[:50000]

In [19]:
bøjning = {0: "infinitiv", 1: "førnutidsformen"}
half_scope = int(scope/2)
big_lst = []
output_lst = []


for i in tqdm(range(len(post_pad_result))):
    current_lst = post_pad_result[i]
    for x in range(len(current_lst)):
        current_word = current_lst[x]
        try: stemmed_word = nutids_r_stem[current_word]
        except: continue
        current_dataset = current_lst[x-half_scope:x+half_scope+1]
        try: current_dataset[half_scope] = stemmed_word
        except: print(current_dataset); continue
        big_lst.append((" ".join(current_dataset)).lower())
        if nutids_r[stemmed_word][0] == current_word:
            output = 0
        elif nutids_r[stemmed_word][1] == current_word:
            output = 1
        else:
            print("ERROR")
        output_lst.append(output)


100%|██████████| 50000/50000 [00:01<00:00, 37801.45it/s]


In [20]:
df = pd.DataFrame()
df["comment_text"] = big_lst
df["label"] = output_lst

print(len(df))
df = df[:6000000]

def distribution(df):
    print(df["label"].value_counts())

distribution(df)
print(len(df))
header = ["comment_text", "label"]
df.to_csv("../Datasets/EuroparlNutidsr.csv", encoding="UTF-8", index=False, sep=";")

143961
0    92156
1    51805
Name: label, dtype: int64
143961


--- 
### Making testset for nutids-r module

In [1]:
import os
import pandas as pd
from tqdm import tqdm
import pickle

In [2]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r_bøjninger.pickle", "rb") as f:
    nutids_r_bøjninger = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [19]:
correct_sentences = []
wrong_sentences = []

for line in tqdm(lines[:1000]):
    line = line.strip("\n")
    words = line.split()
    for i, word in enumerate(words):
        symbol = ""
        if word[-1] in ".,?!":
            symbol = word[-1]
            word = word[:-1]
        try: stemmed = nutids_r_stem[word]
        except: continue
        if nutids_r_bøjninger[stemmed][0] == word:
            words[i] = nutids_r_bøjninger[stemmed][1] + symbol
        else:
            words[i] = nutids_r_bøjninger[stemmed][0] + symbol
    new_line = " ".join(words)
    if line != new_line:
        wrong_sentences.append(new_line)
        correct_sentences.append(line)

100%|██████████| 1000/1000 [00:00<00:00, 51480.28it/s]


In [20]:
df = pd.DataFrame()
df["wrong"] = wrong_sentences
df["correct"] = correct_sentences
df.to_csv("../Datasets/EuroparlNutidsr_testset.csv", encoding="UTF-8", index=False, sep=";")